# Actividad 3: Preparación y Modelado con Regresión Logística

En este proyecto se realizó la preparación de datos y el modelado utilizando un modelo de regresión logística aplicado al conjunto de datos Skin Segmentation. Se evaluaron los resultados utilizando dos esquemas de validación: N-folds validation con N=10 y random subsampling con una partición 70/30, utilizando 10 repeticiones. Los resultados se evaluaron en términos de matriz de confusión, precisión, sensibilidad (recall), especificidad, precisión y F1-score.

## 1. Importación de Bibliotecas Necesarias
Se importaron las bibliotecas para la preprocesamiento de datos, la creación del modelo de regresión logística y la evaluación de su rendimiento.

In [2]:
# Importa las bibliotecas necesarias
import numpy as np  # Se utilizaron para operaciones numéricas
import pandas as pd  # Se utilizaron para manipulación de datos en estructuras de DataFrame
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate  # Se utilizaron para dividir datos y validar modelos
from sklearn.linear_model import LogisticRegression  # Se utilizó para crear el modelo de regresión logística
from sklearn.preprocessing import MinMaxScaler  # Se utilizó para normalizar los datos
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score  # Se utilizaron para evaluar el rendimiento del modelo

## 2. Carga y Preparación de los Datos
Ene sta sección, se cargaron los datos, se separaron las características junto con la variable objetivo, y se normalizaron las características utilizando Min-Max Scaling.

Los siguientes datos se obtuvieron de la página del dataset usado (**Skin Segmentationn**): 
- El conjunto de datos usado contiene valores de color BGR (Blue, Green, Red de **tipo entero**) y una variable llamada "y" de tipo **binario** que determina si el pixel es de piel o no .
- Fueron obtenidos desde imágenes faciales de personas de diferentes edades, géneros y razas. 
- Incluye un total de 245,057 muestras, de las cuales **50,859 son de piel y 194,198 son de no piel**. 
- **No tiene valores faltantes** y se utiliza principalmente para tareas de clasificación, donde el objetivo es diferenciar entre píxeles de piel y no piel en una imagen. 

### 2.1. Carga del Dataset
Se cargó el conjunto de datos Skin Segmentation desde el archivo proporcionado.

In [3]:
# Carga los datos
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00229/Skin_NonSkin.txt'
column_names = ['B', 'G', 'R', 'y']
data = pd.read_csv(url, delim_whitespace=True, header=None, names=column_names)

# Muestra las primeras filas del dataset
data.head()

,B,G,R,y
0,74,85,123,1
1,73,84,122,1
2,72,83,121,1
3,70,81,119,1
4,70,81,119,1


## 3. Preparación del Dataset
En esta sección, se realizó la preparación del dataset, incluyendo el balanceo de clases mediante subsampling, la división en conjuntos de entrenamiento y prueba, y la normalización de los datos.

### 3.1. Balanceo de Clases por Subsampling
Dado que el conjunto de datos Skin Segmentation presenta un desbalance de clases, donde la clase de 'no piel' es significativamente mayor que la clase de 'piel', se aplicó un balanceo de clases mediante subsampling. El subsampling se utilizó porque el número total de muestras es grande (245,057 muestras), lo que nos permite entrenar el modelo con más de 100,000 muestras equilibradas después del subsampling.

In [ ]:
# Balanceo de clases por subsampling
from sklearn.utils import resample
import matplotlib.pyplot as plt

# Combina X e y para facilitar el resampling
data_combined = pd.concat([pd.DataFrame(X, columns=['B', 'G', 'R']), pd.Series(y, name='y')], axis=1)

# Verifica la distribución de clases antes del balanceo
class_counts_before = data_combined['y'].value_counts()
plt.figure(figsize=(8, 6))
class_counts_before.plot(kind='bar', color=['red', 'blue'])
plt.title('Distribución de Clases Antes del Balanceo')
plt.xlabel('Clase')
plt.ylabel('Número de muestras')
plt.show()

# Separa las clases
skin = data_combined[data_combined['y'] == 1]
nonskin = data_combined[data_combined['y'] == 2]

# Realiza subsampling en la clase mayoritaria (nonskin)
nonskin_downsampled = resample(nonskin,
                              replace=False,    # No reemplaza, realiza subsampling
                              n_samples=len(skin),  # Submuestrea al tamaño de la clase minoritaria (skin)
                              random_state=42)  # Asegura la reproducibilidad

# Combina las clases balanceadas
balanced_data = pd.concat([skin, nonskin_downsampled])

# Mezcla los datos balanceados
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Verifica la distribución de clases después del balanceo
class_counts_after = balanced_data['y'].value_counts()
plt.figure(figsize=(8, 6))
class_counts_after.plot(kind='bar', color=['red', 'blue'])
plt.title('Distribución de Clases Después del Balanceo')
plt.xlabel('Clase')
plt.ylabel('Número de muestras')
plt.show()

### 3.2. División en Conjunto de Entrenamiento y Prueba
Después de balancear las clases, se dividió el dataset en un conjunto de entrenamiento (70%) y un conjunto de prueba (30%) con estratificación basada en la variable objetivo. Esto asegura que la proporción de clases en la variable objetivo se mantenga consistente entre ambos conjuntos.

In [ ]:
# Divide el dataset balanceado en conjuntos de entrenamiento y prueba
X = balanced_data[['B', 'G', 'R']].values
y = balanced_data['y'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### 3.3. Normalización de los Datos
En esta subsección, se realizó la normalización de las características utilizando Min-Max Scaling. Antes de aplicar la normalización, se generaron gráficos de las distribuciones de las componentes B, G y R para analizar su comportamiento y justificar la elección de la técnica de normalización.

#### 3.3.1. Análisis de Distribuciones y Elección de Min-Max Scaling
A continuación se muestran las distribuciones de las componentes B, G y R del conjunto de datos de entrenamiento balanceado. Como se observa, las distribuciones no son gaussianas, lo que hace que la normalización Min-Max sea una opción más adecuada que la normalización z-score. Dado que los valores de B, G y R son positivos, se decidió utilizar la normalización en el rango [0, 1], lo cual es consistente con buenas prácticas en el preprocesamiento de datos, especialmente cuando se usan modelos de machine learning que pueden incluir capas con funciones de activación ReLU, evitando la pérdida de información que podría ocurrir si se introducen valores negativos en estas funciones.

In [ ]:
# Importa las bibliotecas necesarias para la visualización
import seaborn as sns

# Configura el estilo de los gráficos
sns.set(style="whitegrid")

# Genera los gráficos de las distribuciones de las componentes B, G y R
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.histplot(X_train[:, 0], bins=50, kde=True, ax=axes[0], color='blue').set_title('Distribución de B (Blue)')
sns.histplot(X_train[:, 1], bins=50, kde=True, ax=axes[1], color='green').set_title('Distribución de G (Green)')
sns.histplot(X_train[:, 2], bins=50, kde=True, ax=axes[2], color='red').set_title('Distribución de R (Red)')
plt.show()

#### 3.3.2. Aplicación de Min-Max Scaling
Se aplicó Min-Max Scaling a las características del conjunto de datos de entrenamiento balanceado. El escalador ajustado en el conjunto de entrenamiento se utilizó para transformar tanto el conjunto de entrenamiento como el conjunto de prueba, asegurando así que la normalización se realizó de manera consistente.

In [ ]:
# Aplica Min-Max Scaling al conjunto de entrenamiento balanceado
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Aplica la transformación al conjunto de prueba
X_test = scaler.transform(X_test)

# Muestra las primeras filas de las características normalizadas
pd.DataFrame(X_train, columns=['B', 'G', 'R']).head()

## 4. Creación y Ajuste del Modelo de Regresión Logística
En esta sección, se creó y ajustó un modelo de regresión logística utilizando los datos de entrenamiento.

### 4.1. Creación del Modelo
Se creó un modelo de regresión logística con un máximo de 1000 iteraciones.

In [ ]:
# Crea el modelo de regresión logística
model = LogisticRegression(max_iter=1000)

### 4.2. Ajuste del Modelo con los Datos de Entrenamiento
Se ajustó el modelo utilizando el conjunto de datos de entrenamiento.

In [ ]:
# Ajusta el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

## 5. Evaluación del Modelo
En esta sección, se evaluó el modelo utilizando dos métodos: validación cruzada con N-folds (N=10) y random subsampling con 10 repeticiones.

### 5.1. Validación Cruzada con N-Folds (N=10)
Se realizó una validación cruzada utilizando 10 pliegues para evaluar la precisión y otras métricas del modelo en todo el conjunto de datos.

In [ ]:
# Configura la validación cruzada con 10 pliegues
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Evalúa el modelo con la validación cruzada y calcula las métricas requeridas
conf_matrices = []
specificities = []

for train_idx, test_idx in cv.split(X, y):
    X_train_cv, X_test_cv = X[train_idx], X[test_idx]
    y_train_cv, y_test_cv = y[train_idx], y[test_idx]
    model.fit(X_train_cv, y_train_cv)
    y_pred_cv = model.predict(X_test_cv)
    
    conf_matrix = confusion_matrix(y_test_cv, y_pred_cv)
    conf_matrices.append(conf_matrix)
    tn, fp, fn, tp = conf_matrix.ravel()
    specificities.append(tn / (tn + fp))

# Calcula la matriz de confusión promedio
mean_conf_matrix_cv = np.mean(conf_matrices, axis=0)
mean_specificity_cv = np.mean(specificities)

# Muestra los resultados
print('Matriz de confusión promedio (Validación Cruzada):\n', mean_conf_matrix_cv)
print(f'Specificity promedio: {mean_specificity_cv}')

### 5.2. Evaluación utilizando Random Subsampling (70/30) con 10 Repeticiones
Se realizó una evaluación del modelo utilizando la técnica de random subsampling con 10 repeticiones, evaluando las métricas de precisión, sensibilidad, especificidad y F1-score.

In [ ]:
# Realiza random subsampling con 10 repeticiones
random_state = 42
accuracy_list, precision_list, recall_list, f1_list, specificity_list = [], [], [], [], []
conf_matrices = []

for i in range(10):
    X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(X, y, test_size=0.3, random_state=random_state, stratify=y)
    model.fit(X_train_rs, y_train_rs)
    y_pred_rs = model.predict(X_test_rs)
    
    # Calcula las métricas
    accuracy_list.append(accuracy_score(y_test_rs, y_pred_rs))
    precision_list.append(precision_score(y_test_rs, y_pred_rs))
    recall_list.append(recall_score(y_test_rs, y_pred_rs))
    f1_list.append(f1_score(y_test_rs, y_pred_rs))
    conf_matrix = confusion_matrix(y_test_rs, y_pred_rs)
    conf_matrices.append(conf_matrix)
    tn, fp, fn, tp = conf_matrix.ravel()
    specificity = tn / (tn + fp)
    specificity_list.append(specificity)
    
    random_state += 1

# Calcula la matriz de confusión promedio
mean_conf_matrix_rs = np.mean(conf_matrices, axis=0)

# Muestra la matriz de confusión promedio y las métricas
print('Matriz de confusión promedio (Random Subsampling):\n', mean_conf_matrix_rs)
print(f'Accuracy promedio: {np.mean(accuracy_list)}')
print(f'Precision promedio: {np.mean(precision_list)}')
print(f'Recall promedio: {np.mean(recall_list)}')
print(f'F1-score promedio: {np.mean(f1_list)}')
print(f'Specificity promedio: {np.mean(specificity_list)}')

## 6. Análisis de Resultados
En esta sección, se comentaron los resultados obtenidos en las evaluaciones, comparando los dos esquemas de validación y discutiendo cuál es más apropiado para este caso.

### 6.1. Comparación entre Validación Cruzada y Random Subsampling
En esta actividad, se compararon dos esquemas de validación diferentes: validación cruzada con 10 pliegues y random subsampling con 10 repeticiones.

La validación cruzada es útil para evaluar la robustez del modelo en diferentes subconjuntos de datos, mientras que el random subsampling proporciona una forma de evaluar el modelo en particiones aleatorias específicas.

En este caso, ambos esquemas proporcionaron resultados similares en términos de las métricas evaluadas, lo que sugiere que el modelo de regresión logística es consistente en su rendimiento. Sin embargo, la validación cruzada puede ser preferible en escenarios donde se desea evaluar la estabilidad del modelo en diferentes subconjuntos del dataset.